In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("../input/all-labelled/all_labelled.tsv", delimiter="\t")
validation=pd.read_csv("../input/test-val/val_data.tsv", delimiter="\t")
cl=pd.read_csv("../input/top-losses-cl/top_losses_cl.tsv", delimiter= "\t")

In [ ]:
train['class']=train['class'].astype('bool')
validation['class']=validation['class'].astype('bool')
cl['class']=cl['class'].astype('bool')

In [ ]:
print(train[train['uid'].isin(cl['uid'])])

In [ ]:
train.loc[train['uid'].isin(cl['uid']), 'class'] = ~(train.loc[train['uid'].isin(cl['uid']), 'class'] )

In [ ]:
print(train[train['uid'].isin(cl['uid'])])

In [ ]:
# Pre-process train data 
train.dropna(inplace=True)
train=train.drop_duplicates()

In [ ]:
!pip install -q ktrain 

In [ ]:
# Import required libraries 
import tensorflow as tf 
import ktrain 
from ktrain import text 
from sklearn.model_selection import train_test_split
# Enable AMP
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [ ]:
# Set random seed
import os
import random
def reset_random_seeds(seed=3):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
reset_random_seeds() 

In [ ]:
X_train=train['abstract'].to_numpy()
y_train=train['class'].to_numpy()

In [ ]:
X_val=validation['abstract'].to_numpy()
y_val=validation['class'].to_numpy()

In [ ]:
t_mod = text.Transformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', maxlen=500, class_names = [0,1])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef

In [ ]:
trn=t_mod.preprocess_train(X_train, y_train)
val = t_mod.preprocess_test(X_val, y_val)
model=t_mod.get_classifier()
learner=ktrain.get_learner(model,train_data=trn, val_data=val, batch_size=16)

In [ ]:
learning_rate= 2e-5 
epochs=3 
learner.fit_onecycle(learning_rate,epochs)

In [ ]:
predictor=ktrain.get_predictor(learner.model, preproc=t_mod)
predictions=predictor.predict((X_val))
print(classification_report((y_val), predictions))
print(matthews_corrcoef((y_val), predictions))

In [ ]:
predictor.save('./')